In [19]:
import pandas as pd
import sys
import torch

from datetime import datetime
from pprint import pprint


if 'src' not in sys.path:
    sys.path.append('src')

%reload_ext autoreload
%autoreload 2

from explore.data import DetectorDataProvider, LookUpTable

### Read data from the data set

Get list of all sections

In [15]:
lut = LookUpTable('Data')
sections = set()
for inter in lut.list_intersections():
    detectors = lut.get_detectors_on(inter)
    for sec in detectors[['Starting Intersection', 'Ending Intersection']].values:
        sections.add(tuple(sorted(sec)))
pprint(list(sections)[:10])

c:\Users\19ak9\TUM\IDP\src\explore\data.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.lookup_table = self.lookup_table[


[('8602', '8604'),
 ('3060', 'RL'),
 ('8402', 'Am Heidgraben'),
 ('1012', '26&8001'),
 ('6022', 'Network Boundary'),
 ('3022', '3040'),
 ('2010', '3021'),
 ('3100', 'Lena Christ strasse'),
 ('24', '4020'),
 ('8005', '8006')]


Extract list of detectors for each section

In [16]:
lut = LookUpTable('Data')
int_det = []
for int_1, int_2 in sections:
    det_1_2, det_2_1 = lut.get_detectors_between(int_1, int_2)
    int_det.append((int_1, int_2, det_1_2))
    int_det.append((int_2, int_1, det_2_1))
int_det = pd.DataFrame(int_det, columns=['Start', 'End', 'Detectors'])
int_det

c:\Users\19ak9\TUM\IDP\src\explore\data.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.lookup_table = self.lookup_table[


,Start,End,Detectors
0,8602,8604,[]
1,8604,8602,[2(DA2)]
2,3060,RL,[]
3,RL,3060,[3(DB1)]
4,8402,Am Heidgraben,[]
...,...,...,...
467,Regensburger Strasse,2050,[9(DD1)]
468,8006,8007,[]
469,8007,8006,[3(DB1)]
470,3140,Lentinger Strasse,[]


Extract data from January to August and accumulate counts by section

In [29]:
ddp = DetectorDataProvider('Data')

In [30]:
START_DATE = datetime(2021, 1, 1)
END_DATE = datetime(2021, 12, 1)

lens = set()
def get_count(section_end, detectors):
    section_data = ddp.get_data_for_period(section_end, START_DATE, END_DATE)
    for col in section_data.columns:
        section_data[col] = pd.to_numeric(section_data[col], errors='coerce')
    try:
        return list(section_data[detectors].fillna(0).sum(axis=1, numeric_only=True).cumsum())
    except KeyError:
        print(section_end, section_data.columns, detectors)
int_det['Counts'] = int_det.apply(lambda sec: get_count(sec['End'], sec['Detectors']), axis=1)

int_det.to_hdf('int_det_excluded_missing.hdf', 'int_det')
int_det

96 DetCount_20210101.csv
100 DetCount_20211031.csv


KeyboardInterrupt: 

### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [22]:
int_det['Counts'].apply(len)

0      32068
1      32068
2      32064
3      32068
4      32064
       ...  
467    32068
468    32064
469    32068
470    32064
471    32068
Name: Counts, Length: 472, dtype: int64

In [20]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
torch.save(mat_q, 'mat_q_excluded_missing.pt')
mat_q.shape

ValueError: expected sequence of length 32068 at dim 1 (got 32064)